In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [2]:
df_elering = pd.read_csv('../Data_folder/elering_da_data.csv', sep=';')

In [3]:

target_mae = 5.444
target_rmse = 7.152

results_3 = []
best_metrics = {'rmse': np.inf, 'mae': np.inf}
temp_best_metrics = {'mae': 100, 'rmse': 100}

best_achieved_mae = 10e6
best_achieved_rmse = 10e6

DA_price = df_elering['DA_price'].values

global_score = 10e6



np.random.seed(31)
for proportionality_factor in np.arange(0.00, 0.101, 0.001):
    scaling_factor = np.abs(DA_price) * proportionality_factor

    for scale_noise in np.arange(0.00, 0.181, 0.001):
        variable_noise = np.random.normal(0, scale_noise, size=DA_price.shape[0]) * scaling_factor

        for slope_scale in np.arange(0.00, 0.081, 0.001):
            slope = np.random.normal(1, slope_scale, size=DA_price.shape[0])
            predictions = DA_price * slope + variable_noise

            achieved_MAE = mean_absolute_error(DA_price, predictions)
            achieved_RMSE = np.sqrt(mean_squared_error(DA_price, predictions))

            if achieved_MAE > target_mae and achieved_RMSE > target_rmse:

                local_score = (np.abs(achieved_MAE - target_mae)*1.3 + np.abs(achieved_RMSE - target_rmse))/2

                if local_score < global_score:
                    
                    best_preds = predictions
                    best_achieved_mae = np.abs(achieved_MAE - target_mae)
                    best_achieved_rmse = np.abs(achieved_RMSE - target_rmse)
                    global_score = local_score

                    if local_score < 0.5:
                        results_3.append({
                                'proportionality_factor': round(proportionality_factor, 4),
                                'scale_noise': round(scale_noise, 4),
                                'slope_scale': round(slope_scale, 4),
                                'intercept': 0,  # Intercept is always 0 in your setup
                                'MAE': round(achieved_MAE, 4),
                                'RMSE': round(achieved_RMSE, 4)
                            })

results_df_3 = pd.DataFrame(results_3)
print(len(results_df_3))
results_df_3.head()


12


,proportionality_factor,scale_noise,slope_scale,intercept,MAE,RMSE
0,0.000,0.000,0.074,0,5.4450,7.9629
1,0.000,0.003,0.076,0,5.4550,7.9354
2,0.000,0.091,0.075,0,5.4511,7.9264
3,0.000,0.131,0.075,0,5.4668,7.9058
4,0.001,0.085,0.076,0,5.4535,7.9002


In [4]:
df_elering['fake_pred'] = best_preds

df_elering['fake_pred'] = [round(i, 2) for i in df_elering['fake_pred']]

df_elering
df_elering.to_csv('da_and_fake_pred.csv', sep=';')
